In [1]:
# Dependencies and setup
import pandas as pd
from pathlib import Path
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
# File to load
to_load = Path("cancer_patient_datasets.csv")
# Create the dataframe
nn_df = pd.read_csv(to_load)

In [3]:
# drop unnecessary columns index and Patient Id
nn_df = nn_df.drop(columns=['index', 'Patient Id'])

In [4]:
# convert low, medium, high to numeric
nn_df.Level = nn_df.Level.replace("Low", 0)
nn_df.Level = nn_df.Level.replace("Medium", 1)
nn_df.Level = nn_df.Level.replace("High", 2)

nn_df.Level = nn_df.Level.astype("int64")
# create target and features array
X = nn_df.drop("Level", axis = 1)
y = pd.get_dummies(nn_df["Level"])

In [5]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:

def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    # Correctly setting up the choice for activation functions
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])
    
    # Correcting the input shape parameter
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=1, max_value=128, step=32),
        activation=activation, 
        input_shape=(23,)
    ))
    
    # Adding a range for the number of additional layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=1, max_value=128, step=32),
            activation=activation
        ))
    
    # Assuming 3 classes in the output layer
    nn_model.add(tf.keras.layers.Dense(units=3, activation="softmax"))
    nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model


In [8]:
from keras_tuner import Hyperband, Objective

tuner = Hyperband(
    create_model,
    objective=Objective("val_accuracy", direction="max"),
    max_epochs=100,
    hyperband_iterations=2,
    directory='hyperband',
    project_name='cancer_classification'
)
tuner.search_space_summary()

Reloading Tuner from hyperband\cancer_classification\tuner0.json
Search space summary
Default search space size: 9
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
first_units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 6, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_val

In [9]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test_scaled, y_test))

In [10]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 65,
 'num_layers': 1,
 'units_0': 97,
 'units_1': 65,
 'units_2': 65,
 'units_3': 97,
 'units_4': 65,
 'units_5': 97,
 'tuner/epochs': 34,
 'tuner/initial_epoch': 12,
 'tuner/bracket': 4,
 'tuner/round': 3,
 'tuner/trial_id': '0392'}

In [11]:
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

C:\Users\GamingPC\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\GamingPC\anaconda3\Lib\site-packages\keras_tuner\src\tuners\hyperband.py:435: UserWarning: Model 'sequential' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  model.build_from_config(
C:\Users\GamingPC\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading f

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0232  
Loss: 0.021024776622653008
Accuracy: 1.0


In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(shape=(23,)),  # Assuming 23 input features
    tf.keras.layers.Dense(65, activation='relu'),# First dense layer with 65 units
    tf.keras.layers.Dense(95, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train_scaled,y_train, epochs=34, validation_data=(X_test_scaled,y_test),verbose=2)

Epoch 1/34
24/24 - 1s - 28ms/step - accuracy: 0.6947 - loss: 0.8116 - val_accuracy: 0.8240 - val_loss: 0.5286
Epoch 2/34
24/24 - 0s - 2ms/step - accuracy: 0.8653 - loss: 0.4078 - val_accuracy: 0.8640 - val_loss: 0.3327
Epoch 3/34
24/24 - 0s - 2ms/step - accuracy: 0.9173 - loss: 0.2751 - val_accuracy: 0.9360 - val_loss: 0.2322
Epoch 4/34
24/24 - 0s - 2ms/step - accuracy: 0.9667 - loss: 0.2006 - val_accuracy: 0.9840 - val_loss: 0.1738
Epoch 5/34
24/24 - 0s - 2ms/step - accuracy: 0.9773 - loss: 0.1534 - val_accuracy: 0.9840 - val_loss: 0.1351
Epoch 6/34
24/24 - 0s - 2ms/step - accuracy: 0.9773 - loss: 0.1203 - val_accuracy: 0.9840 - val_loss: 0.1074
Epoch 7/34
24/24 - 0s - 2ms/step - accuracy: 0.9747 - loss: 0.0970 - val_accuracy: 0.9840 - val_loss: 0.0869
Epoch 8/34
24/24 - 0s - 2ms/step - accuracy: 0.9893 - loss: 0.0798 - val_accuracy: 1.0000 - val_loss: 0.0709
Epoch 9/34
24/24 - 0s - 2ms/step - accuracy: 0.9973 - loss: 0.0662 - val_accuracy: 1.0000 - val_loss: 0.0594
Epoch 10/34
24/24 

In [25]:
model.save('Optimised_model.h5')

In [16]:
# Export our model to HDF5 file
# Define the file path for saving the model
filepath = "LungCancer.h5"

# Save the model to HDF5 format
model.save(filepath)